In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/MTP phase 2/copy_DRT_No Claim Certificate.xlsx")
df.head(10)

,text,label
0,"On 21.09.2012, the Contractee Company submitte...",1.0
1,"Subsequently, on 24.10.2012, the Contractee Co...",1.0
2,"On 19.06.2013, the appellant-Contractor issued...",1.0
3,This appeal is directed against the final judg...,0.0
4,Respondent No. 1-the Contractee Company was aw...,0.0
5,The total contract value as per the Letter of ...,0.0
6,"Thereafter, on 10.10.2012, the appellant herei...",0.0
7,"The Contractee-Company, vide letter dated 12.0...",0.0
8,"After several communication in writing, the ap...",0.0
9,"Vide letter dated 14.09.2013, the contractee-C...",0.0


In [ ]:
sent = df['text'][2]
sent

'On 19.06.2013, the appellant-Contractor issued a Completion Certificate stating that the works awarded under the Contract have been executed and completed in all respects and no claim certificate has also been submitted by the Contractee-Company'

In [ ]:
lst = ['no claim certificate']
label = 0
for i in sent.split():
  if i in lst:
    label = 1


In [ ]:
label

0

In [ ]:
df['TEXT']

KeyError: ignored

In [ ]:
df['class']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
250    1.0
251    1.0
252    1.0
253    1.0
254    1.0
Name: class, Length: 255, dtype: float64

In [ ]:
import string

In [ ]:
def text_cleaning(a):
  remove_punctuation = [char for char in a if char not in string.punctuation]
  remove_punctuation = ''.join(remove_punctuation)
  return [word for word in remove_punctuation.split()] #if word.lower() not in stopwords.words('english')]

In [ ]:
print(df.iloc[:,0].apply(text_cleaning))

0      [This, appeal, is, directed, against, the, fin...
1      [A, petition, under, Section, 34, of, the, Arb...
2      [At, the, outset, it, is, observed, that, thou...
3      [This, is, an, appeal, under, section, 37, of,...
4      [This, order, shall, dispose, of, the, above, ...
                             ...                        
250    [Milestone, 2, Complete, length, of, 25, Km, o...
251    [Milestone, 3, Remaining, length, including, a...
252    [The, period, of, contract, was, 12, months, f...
253    [In, terms, of, the, contract, three, indents,...
254    [The, brief, facts, of, the, case, is, that, t...
Name: TEXT, Length: 255, dtype: object


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer = text_cleaning).fit(df['TEXT'])

In [ ]:
title_bow = bow_transformer.transform(df['TEXT'])

In [ ]:
x = title_bow.toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(title_bow)
title_tfidf = tfidf_transformer.transform(title_bow)

In [ ]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(title_tfidf, df['class'], test_size = 0.2, random_state = 0)

In [ ]:
model1 = MultinomialNB().fit(X_train, y_train)
model2 = BernoulliNB().fit(X_train, y_train)
model3 = LinearSVC().fit(X_train, y_train)
model4 = LogisticRegression().fit(X_train, y_train)
model5 = KNeighborsClassifier().fit(X_train, y_train)
model6 = DecisionTreeClassifier().fit(X_train, y_train)

In [ ]:
all_pred1 = model1.predict(X_test)
all_pred2 = model2.predict(X_test)
all_pred3 = model3.predict(X_test)
all_pred4 = model4.predict(X_test)
all_pred5 = model5.predict(X_test)
all_pred6 = model6.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, all_pred3)

array([[15,  3],
       [ 2, 31]])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print('MultinomialNB:',accuracy_score(y_test,all_pred1))
print('BernoulliNB:',accuracy_score(y_test,all_pred2))
print('LinearSVC:',accuracy_score(y_test,all_pred3))
print('LogisticRegression:',accuracy_score(y_test,all_pred4))
print('KNeighborsClassifier:',accuracy_score(y_test,all_pred5))
print('DecisionTreeClassifier:',accuracy_score(y_test,all_pred6))

MultinomialNB: 0.9019607843137255
BernoulliNB: 0.8823529411764706
LinearSVC: 0.9019607843137255
LogisticRegression: 0.8823529411764706
KNeighborsClassifier: 0.9215686274509803
DecisionTreeClassifier: 0.9215686274509803
